## Ecosystem Classifier

The EcosystemClassifier model is a voting classifier composed of Random Forests. It encompasses the following methods:

1. Predict method
2. Predict probability method
3. Feature importance evaluation

In [1]:
import numpy as np
import pandas as pd

In [2]:
class EcosystemClassifier(object):
    __slots__ = ("weights",
                 "models",
                 "reports")

    def __init__(self,
                 weights: pd.Series,
                 models: pd.Series,
                 reports: pd.Series):
        """
        Constructor method for the class `EcosystemClassifier`.
        :param weights: Weights to each Random Forest for the Voting Classifier.
        :param models: List of Random Forests.
        :reports: List of OptimizeResult objects.
        """
        self.weights = weights
        self.models = models
        self.reports = reports

    def predict_proba(self, 
                      x: np.array
                      ) -> np.array:
        """
        Predict probability method for x. The predicted class of an input sample is a weighted vote by the forests in the ecosystem.
        :input x: The input samples.
        :return: The predicted classes probabilities.
        """
        predict = self.models.apply(func=lambda model: np.array([proba[1] for proba in model.predict_proba(x)]))
        predict = (predict * self.weights / self.weights.sum()).sum()
        return predict
        
    def predict(self, 
                x: np.array
                ) -> np.array:
        """
        Predict method for x. The predicted class of an input sample is a weighted vote by the forests in the ecosystem.
        :input x: The input samples.
        :return: The predicted classes.
        """
        return self.predict_proba(x) > .5

    def feature_importance(self,
                           feature_names: list[str]
                           ) -> pd.DataFrame:
        """
        Feature importance method. The importances are a weighted vote by the forests in the ecosystem.
        :input feature_names: List containing the names of the features.
        :return: Feature importance DataFrame.
        """
        feature_importance = self.models.apply(func=lambda x: x.feature_importances_)
        df = pd.DataFrame(data=(feature_importance * self.weights).sum() / self.weights.sum(),
                          index=feature_names, 
                          columns=["relevance"])
        return df.sort_values(by="relevance", ascending=False)